Let's make a cool tutorial!

In [6]:
from astropy.utils.data import download_file
from astropy.io import fits

In [43]:
import os

if not os.path.isfile('TAN_C14.fits'):
    hi_datafile = download_file('http://cdsarc.u-strasbg.fr/vizier/ftp/cats/J/A+A/594/A116/CUBES/GAL/TAN/TAN_C14.fits',
                       cache=True)
else:
    hi_data = 'TAN_C14.fits'

#hi_datafile = download_file('http://cdsarc.u-strasbg.fr/vizier/ftp/cats/J/A+A/594/A116/CUBES/GAL/TAN/TAN_C14.fits', 
                        #cache=True )

hi_data = fits.open(hi_datafile)

In [44]:
from spectral_cube import SpectralCube


In [45]:
cube = SpectralCube.read(hi_data)
hi_data.close()

/Users/DK/anaconda/envs/dk/lib/python3.6/site-packages/spectral_cube/spectral_cube.py:238: UserWarning: radio_beam is not installed. No beam can be created.
  warnings.warn("radio_beam is not installed. No beam "
/Users/DK/anaconda/envs/dk/lib/python3.6/site-packages/spectral_cube/spectral_cube.py:246: UserWarning: Could not parse beam information from header.  Exception was: UnboundLocalError("local variable 'Beam' referenced before assignment",)
  "  Exception was: {0}".format(ex.__repr__()))


In [11]:
cube

SpectralCube with shape=(933, 277, 269) and unit=K:
 n_x:    269  type_x: GLON-TAN  unit_x: deg    range:   286.707162 deg:  320.913024 deg
 n_y:    277  type_y: GLAT-TAN  unit_y: deg    range:   -51.659734 deg:  -28.394756 deg
 n_s:    933  type_s: VRAD      unit_s: m / s  range:  -600207.218 m / s:  600409.133 m / s

In [13]:
from astropy import units as u
import astropy.coordinates as coord
from astroquery.esasky import ESASky

In [34]:
result = ESASky.query_region_maps('SMC', radius = 1*u.deg)
print(result)

TableList with 9 tables:
	'0:INTEGRAL' with 8 column(s) and 18 row(s) 
	'1:XMM-EPIC' with 9 column(s) and 107 row(s) 
	'2:CHANDRA' with 10 column(s) and 136 row(s) 
	'3:SUZAKU' with 7 column(s) and 72 row(s) 
	'4:XMM-OM-OPTICAL' with 10 column(s) and 7 row(s) 
	'5:XMM-OM-UV' with 10 column(s) and 47 row(s) 
	'6:HST' with 11 column(s) and 3505 row(s) 
	'7:HERSCHEL' with 11 column(s) and 24 row(s) 
	'8:ISO' with 7 column(s) and 47 row(s) 


In [39]:
result['HERSCHEL']['filter']

"70, 160"
"70, 160"
"100, 160"
"100, 160"
"70, 160"
"70, 160"
"70, 160"
"70, 160"
"70, 160"
"70, 160"
"70, 160"


In [38]:
result['HERSCHEL'].show_in_notebook()

In [30]:
print(result)

TableList with 9 tables:
	'0:INTEGRAL' with 8 column(s) and 18 row(s) 
	'1:XMM-EPIC' with 9 column(s) and 107 row(s) 
	'2:CHANDRA' with 10 column(s) and 136 row(s) 
	'3:SUZAKU' with 7 column(s) and 72 row(s) 
	'4:XMM-OM-OPTICAL' with 10 column(s) and 7 row(s) 
	'5:XMM-OM-UV' with 10 column(s) and 47 row(s) 
	'6:HST' with 11 column(s) and 3505 row(s) 
	'7:HERSCHEL' with 11 column(s) and 24 row(s) 
	'8:ISO' with 7 column(s) and 47 row(s) 


In [25]:
table = result['HERSCHEL']
print(table)

                                                                          postcard_url                                                                           ...
---------------------------------------------------------------------------------------------------------------------------------------------------------------- ...
http://archives.esac.esa.int/hsa/aio/jsp/product.jsp?RETRIEVAL_TYPE=POSTCARD&PROTOCOL=HTTP&OBSERVATION.OBSERVATION_ID=1342225318&INSTRUMENT.INSTRUMENT_NAME=PACS ...
http://archives.esac.esa.int/hsa/aio/jsp/product.jsp?RETRIEVAL_TYPE=POSTCARD&PROTOCOL=HTTP&OBSERVATION.OBSERVATION_ID=1342214770&INSTRUMENT.INSTRUMENT_NAME=PACS ...
http://archives.esac.esa.int/hsa/aio/jsp/product.jsp?RETRIEVAL_TYPE=POSTCARD&PROTOCOL=HTTP&OBSERVATION.OBSERVATION_ID=1342205092&INSTRUMENT.INSTRUMENT_NAME=PACS ...
http://archives.esac.esa.int/hsa/aio/jsp/product.jsp?RETRIEVAL_TYPE=POSTCARD&PROTOCOL=HTTP&OBSERVATION.OBSERVATION_ID=1342205055&INSTRUMENT.INSTRUMENT_NAME=PACS ...
http://arc